In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
quiet_library(ArchR)
quiet_library(cowplot)
# quiet_library(SeuratData)
quiet_library(SeuratDisk)
quiet_library(pheatmap)

In [ ]:
mnp <- readRDS('mnp_analysis_3wnn_updated.rds')

In [ ]:
table(mnp$cell.types)

In [ ]:
thymus <- LoadH5Seurat('/home/jupyter/Thymus_Paper/t_cells_resaved.h5seurat')

In [ ]:
mnp$method <- 'TEA-seq'
mnp$Age <- mnp$pediatric_senior

In [ ]:
table(thymus$Age)

In [ ]:
thymus <- SetIdent(thymus, value = 'Age')
thymus_young <- subset(thymus, idents = c('13y','30m','15m','10m'))

In [ ]:
thymus_naive.list <- lapply(X = list(thymus_young,mnp), FUN = function(x) {
    DefaultAssay(x) <- 'RNA'
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000)
})

In [ ]:
features <- SelectIntegrationFeatures(object.list = thymus_naive.list)

In [ ]:
immune.anchors <- FindIntegrationAnchors(object.list = thymus_naive.list, anchor.features = features)

In [ ]:
# this command creates an 'integrated' data assay
immune.combined <- IntegrateData(anchorset = immune.anchors)

In [ ]:
# specify that we will perform downstream analysis on the corrected data note that the
# original unmodified data still resides in the 'RNA' assay
DefaultAssay(immune.combined) <- "integrated"

# Run the standard workflow for visualization and clustering
immune.combined <- ScaleData(immune.combined, verbose = FALSE)
immune.combined <- RunPCA(immune.combined, npcs = 30, verbose = FALSE)
immune.combined <- RunUMAP(immune.combined, reduction = "pca", dims = 1:20)
immune.combined <- FindNeighbors(immune.combined, reduction = "pca", dims = 1:20)
immune.combined <- FindClusters(immune.combined, resolution = 0.5)

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- DimPlot(immune.combined, reduction = "umap", group.by = "method", shuffle = TRUE)
p2 <- DimPlot(immune.combined, reduction = "umap", group.by = "cell.types", label = TRUE,
    repel = TRUE, shuffle = T)
p3 <- DimPlot(immune.combined, reduction = "umap", group.by = "Age", shuffle = TRUE)
p1 + p2 + p3

In [ ]:
library(scales)

In [ ]:
hue_pal()(20)

In [ ]:
show_col(hue_pal()(20))

# Fig 6E

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(immune.combined, reduction = "umap", group.by = "cell.types", label = TRUE,
        repel = TRUE, shuffle = T, cols = c('#F8766D','#E9842C','#D69100','#BC9D00',
                                            '#9CA700','#6FB000','#00C0B4','#00BDD4',
                                            '#00C08E','#33a02c','#b2df8a','#fdbf6f','#1f78b4',
                                            '#00A7FF','#7F96FF','#BC81FF','#E26EF7',
                                            '#F863DF','#FF62BF','#FF6A9A'))

In [ ]:
# Open a pdf file
pdf("plots/thymus_integrated_umap_May23.pdf", width = 10, height = 8) 
# 2. Create a plot
DimPlot(immune.combined, reduction = "umap", group.by = "cell.types", label = FALSE,
        repel = TRUE, shuffle = T, cols = c('#F8766D','#E9842C','#D69100','#BC9D00',
                                            '#9CA700','#6FB000','#00C0B4','#00BDD4',
                                            '#00C08E','#33a02c','#b2df8a','#fdbf6f','#1f78b4',
                                            '#00A7FF','#7F96FF','#BC81FF','#E26EF7',
                                            '#F863DF','#FF62BF','#FF6A9A'))
# Close the pdf file
dev.off() 

In [ ]:
DefaultAssay(immune.combined) <- 'RNA'
immune.combined <- NormalizeData(immune.combined, assay = 'RNA')
immune.combined <- FindVariableFeatures(immune.combined)
immune.combined <- ScaleData(immune.combined, assay = 'RNA', vars.to.regress = 'method')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
p1 <- FeaturePlot(immune.combined, features = 'rna_RTKN2', order = TRUE) & scale_color_viridis()
p2 <- FeaturePlot(immune.combined, features = 'rna_ZNF683', order = TRUE) & scale_color_viridis()
p3 <- FeaturePlot(immune.combined, features = 'rna_GNG4', order = TRUE) & scale_color_viridis()
p4 <- FeaturePlot(immune.combined, features = 'rna_IL21R', order = TRUE) & scale_color_viridis()
# p1

# Fig 6G

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 16)
plot_grid(p1, p2, p3, p4, ncol = 2)

In [ ]:
# Open a pdf file
pdf("plots/thymus_integrated__markersumap.pdf", width = 16, height = 16) 
# 2. Create a plot
plot_grid(p1, p2, p3, p4, ncol = 2)
# Close the pdf file
dev.off() 

In [ ]:
genes_of_interest <- c('CD8A','CD8B','RTKN2','ZNF683','GNG4','MME','NKG7','PECAM1','IFNG','TBX21','EOMES','KLRB1','KLRC2','KLRC3','KLRD1','CD200','IL21R','FAS')

In [ ]:
DefaultAssay(immune.combined) <- 'RNA'
immune.combined <- NormalizeData(immune.combined, assay = 'RNA')
VariableFeatures(immune.combined) <- genes_of_interest
immune.combined <- ScaleData(immune.combined, assay = 'RNA', vars.to.regress = 'method')

In [ ]:
avg_exp <- AverageExpression(immune.combined, assays = 'RNA', features = genes_of_interest, group.by = 'cell.types', slot = 'scale')

In [ ]:
avg_exp_df <- as.data.frame(avg_exp$RNA)

In [ ]:
head(avg_exp_df)

In [ ]:
paletteLength <- 50
myColor <- colorRampPalette(c("blue", "white", "red"))(paletteLength)
# length(breaks) == length(paletteLength) + 1
# use floor and ceiling to deal with even/odd length pallettelengths
myBreaks <- c(seq(min(avg_exp_df), 0, length.out=ceiling(paletteLength/2) + 1), 
              seq(max(avg_exp_df)/paletteLength, max(avg_exp_df), length.out=floor(paletteLength/2)))

# Fig 6F

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p <- pheatmap(avg_exp_df, scale = 'none', color=myColor, breaks=myBreaks, fontsize = 20)

In [ ]:
save_pheatmap_pdf <- function(x, filename, width=10, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}
save_pheatmap_pdf(p, "plots/thymus_heatmap_May23.pdf")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
DefaultAssay(immune.combined) <- 'integrated'
VlnPlot(immune.combined, features = c('RTKN2','ZNF683','GNG4','MME'), group.by = 'cell.types', ncol = 2, pt.size = 0)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
DotPlot(immune.combined, features = genes_of_interest)

In [ ]:
saveRDS(immune.combined, file = 'updated_mnp_thymus.rds')

In [ ]:
immune.combined <- readRDS(file = 'updated_mnp_thymus.rds')

In [ ]:
sessionInfo()